In [1]:
import re                                       
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer

In [2]:
def lower_case(str):
    return str.lower()

about 30% percent of postings are stop words

In [3]:
def remove_stop_words(str):     # do, are ,...
    stopWords = set(stopwords.words('english')) 
    s = " "
    tokens = word_tokenize(str)
    for t in tokens :
        if not t in stopWords:
            s += t + " "
    return s

In [4]:
def remove_numbers(str):
    return re.sub(r'\d+', '', str)

The part of the following function that was commented on was not effective. So I commented it

In [5]:
def remove_punctuation(str):
    # t=""
    # for s in str.split():
    #     if s[0] != "@" :
    #         t += s + " "
    return re.sub(r'[^\w\s]', '', str)

In [6]:
def remove_whitespaces(str):
    return str.strip()

In [7]:
def stemming(str):    #stemming-> stem    goes->go
    str = word_tokenize(str)
    s = ""
    for word in str:
        s+=PorterStemmer().stem(word)+" "
    return s

read file

In [8]:
import pandas as pd
df = pd.read_csv("SandersPosNeg.csv", header=None, sep="\t" )
df

,0,1
0,0,RT @cjwallace03: So apparently @apple put MB c...
1,0,RT @Jewelz2611 @mashable @apple iphones r 2 ex...
2,0,@mashable @apple iphones r 2 expensive. Most w...
3,0,THiS IS WHAT WiLL KiLL APPLE http://t.co/72Jw4...
4,4,Now all @Apple has to do is get swype on the i...
...,...,...
1219,4,RT @ahhfuckitsguss: #twitter can be so useful ...
1220,4,My 3 biggest obsessions: #twitter #dancemoms a...
1221,0,My mentions aren't showing properly ... PAY AT...
1222,4,#twitter is jumpin as usual :)


preprocessing:

remove_whitespaces() and lower_case() are not effective in TfidfVectorizer


In [9]:
# df[1] = df[1].apply(lower_case)
df[1] = df[1].apply(remove_numbers)                
df[1] = df[1].apply(remove_punctuation)
df[1] = df[1].apply(remove_whitespaces)
df[1] = df[1].apply(remove_stop_words)
df[1] = df[1].apply(stemming)            
df[1]

0       RT cjwallac So appar appl put MB cap sm new up...
1       RT jewelz mashabl appl iphon r expens most wen...
2       mashabl appl iphon r expens most went w htcgal...
3        thi IS what will kill appl httptcojwzc rip appl 
4                   now appl get swype iphon crack iphon 
                              ...                        
1219    RT ahhfuckitsguss twitter use let feel cant re...
1220    My biggest obsess twitter dancemom desperateho...
1221    My mention arent show properli pay attent twit...
1222                                twitter jumpin usual 
1223    My facebook mess I make new one add haha least...
Name: 1, Length: 1224, dtype: object

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer                         # Vectorize fd
from sklearn.model_selection import ShuffleSplit, cross_val_score                   # 10fold cross validation

tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(df[1])
Y = df[0]
X

<1224x3234 sparse matrix of type '<class 'numpy.float64'>'
	with 12335 stored elements in Compressed Sparse Row format>

In [11]:
from sklearn.naive_bayes import MultinomialNB

def naive_bayes(cv):
    clf = MultinomialNB(alpha=0.48)
    naiveBayes = cross_val_score(clf, X, df[0], cv=cv)  #naive_bayes for each permutation
    print(naiveBayes*100)
    print(naiveBayes.mean()*100)

In [12]:
from sklearn.neural_network import MLPClassifier

def mlp(cv):
    clf = MLPClassifier(hidden_layer_sizes=(80,), random_state=0, max_iter=150, alpha=0.001)
    NB_result = cross_val_score(clf, X, df[0], cv=cv)
    print(NB_result*100)
    print(NB_result.mean()*100, "%")

n_splits = k = 10


In [13]:
Random_permutation_cross_validator = ShuffleSplit(n_splits = 10, test_size=0.2, random_state=0)      #10 permutation
naiveBayes = naive_bayes(Random_permutation_cross_validator)

[85.71428571 86.12244898 86.12244898 80.         84.89795918 82.44897959
 83.26530612 86.53061224 83.26530612 80.81632653]
83.91836734693878


MLP:

In [14]:
Random_permutation_cross_validator = ShuffleSplit(n_splits = 10, test_size=0.2, random_state=0)      #10 permutation
mlp(Random_permutation_cross_validator)

[86.93877551 88.16326531 86.12244898 79.59183673 86.53061224 86.12244898
 84.89795918 86.12244898 83.67346939 84.48979592]
85.26530612244899 %


MLP is slower and a bit better. So there is a trade-off between time and accuracy

now another dataset

In [15]:
df = pd.read_csv("OMD.csv", header=None, sep="\t")
d0 = []
d1 = []
for d in df.values.tolist() :
    d0.append(d[0][0])
    d1.append(d[0][2:])

df = pd.DataFrame(list(zip(d0, d1)))
df

,0,1
0,0,Watching by myself #tweetdebate Not drinking ...
1,0,"@ahg3 @MichDot Yeah, slime was actually my sec..."
2,0,"Preparing to have a heart attack #tweetdebate,"
3,0,"no debate moderators under 50, sorry #tweetde..."
4,0,@current Now staring at black screen on http:/...
...,...,...
1901,4,"@Imarilove Yes, He did. I liked the eye contac..."
1902,4,Bloggers right; mccain won because it is his i...
1903,4,"Anyone trying to say John McCain is a liar, su..."
1904,4,CNN post debate polling is saying the debate w...


preprocessing:
Those who commented either had no effect or made it worse

In [16]:

# df[1] = df[1].apply(lower_case)                    
df[1] = df[1].apply(remove_numbers)                
df[1] = df[1].apply(remove_punctuation)
# df[1] = df[1].apply(remove_whitespaces)            
df[1] = df[1].apply(remove_stop_words)
# df[1] = df[1].apply(stemming)                      

In [17]:
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(df[1])
Random_permutation_cross_validator = ShuffleSplit(n_splits = 10, test_size=0.2, random_state=0)

naive bayes:

In [18]:

naiveBayes = naive_bayes(Random_permutation_cross_validator)

[76.43979058 78.53403141 79.05759162 72.77486911 74.34554974 73.29842932
 78.27225131 76.70157068 77.48691099 77.22513089]
76.41361256544504


In [21]:
mlp(Random_permutation_cross_validator)

[74.86910995 78.27225131 79.58115183 76.70157068 76.96335079 74.08376963
 75.91623037 77.22513089 78.27225131 76.96335079]
76.88481675392669 %


Naive Bayes is really faster. and The difference between the accuracy of naive Bayes and MLP is small. So in this case Naive bayes is better